In [4]:
import tensorflow as tf
import numpy as np
from importlib import reload

In [ ]:
graph = tf.Graph()
#define graph
with graph.as_default():
    '''
    define variables
    '''
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

#excute op
with sess.as_default():
    re = sess.run([op])

In [5]:
data = np.load('../train.npz')
x_i = data['xi']
x_v = data['xv']
y = data['y']

In [8]:
config = {
    'feature_size' : 4260,
    'field_size': 24,
    'deep_size'  : [32,32],
    'embedding_size' : 8,
    'lr' : 0.01,
    'l2_reg':0.01,
    'deep_init': None,
    'deep_regularizer' : None,
    'deep_activation' : None,
    'epochs':1,
    'batch_size':32,
    
}

In [99]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
import sys
class DeepFM(BaseEstimator,TransformerMixin):
    
    def __init__(self,feature_size,field_size,embedding_size,deep_size,lr,l2_reg,epochs,batch_size,
                deep_init,deep_regularizer,deep_activation,):
        self.feature_size = feature_size
        self.field_size = field_size
        self.embedding_size = embedding_size
        self.deep_size = deep_size
        self.lr = lr
        self.l2_reg = l2_reg
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.deep_init = tf.initializers.glorot_normal()
        self.deep_regularizer = tf.contrib.layers.l2_regularizer(scale=self.l2_reg)
        self.deep_activation = tf.nn.relu
        self.eval_metric = roc_auc_score
        self.init_graph()
                 
        
    
    def dense_layer(self,inputs,units,bias_init=tf.initializers.constant(0.01) ,activation=tf.nn.relu, name=""):
        # bias 初始化为0.01,保持relu激活
        return tf.layers.dense(inputs = inputs,units = units, kernel_initializer=self.deep_init, use_bias=True,
                               kernel_regularizer=self.deep_regularizer, bias_initializer=bias_init,activation=activation,name=name)
    def init_graph(self):
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.feat_index = tf.placeholder(tf.int32,shape=[None,self.field_size],name='feat_index')
            self.feat_value = tf.placeholder(tf.float32,shape=[None,self.field_size], name = 'feat_value')
            self.labels = tf.placeholder(tf.float32,shape=[None,1],name = 'labels')
            self.train_phase = tf.placeholder(tf.bool,name = 'train_phase')
                 
                 
            #input
            self.weights = {};
            self.weights['feature_embedding'] = tf.Variable(tf.random_normal(shape=[self.feature_size,self.embedding_size],mean=0.0,stddev=0.01,name='feature_embedding'))
             # weights of first_order
            self.weights['feature_bias'] = tf.Variable(tf.random_normal(shape=[self.feature_size,1],mean=0.0,stddev=1.0,name='feature_bias'))
            
            self.feature_embedding = tf.nn.embedding_lookup(self.weights['feature_embedding'],ids = self.feat_index) # batch_size * field_size * embedding_size
            
            
           
            #fm part 
            # first order
            feat_value = tf.reshape(self.feat_value, shape=[-1,self.field_size,1]) #batch_size * field_size * 1
            self.first_order = tf.nn.embedding_lookup(self.weights['feature_bias'],self.feat_index) # batch_size * filed_size * 1
            self.first_order = tf.reduce_sum(tf.multiply(self.first_order,feat_value),axis=2) # batch_size * filed_size (squeeze)
            
            #second_order
            self.embeddings = tf.multiply(self.feature_embedding,feat_value) # batch_size * field_size * embedding_size
                 
            self.square_sum_embed = tf.square(tf.reduce_sum(self.embeddings,axis=1)) # batch_size * embedding_size
            
            self.sum_square_embed = tf.reduce_sum(tf.square(self.embeddings),axis=1) # batch_size * embedding_size
            
            self.second_order = 0.5 * tf.subtract(self.sum_square_embed,self.square_sum_embed)
            
            self.fm_output = tf.concat([self.first_order,self.second_order],axis = 1)
        
            #deep part
            self.deep_input = tf.reshape(self.embeddings,shape=[-1,self.field_size*self.embedding_size])
            self.deep_layer_nums = len(self.deep_size)
            self.deep_layers = {}
            self.deep_layers['layer_0'] = self.deep_input
            for i in range(self.deep_layer_nums):
                self.deep_layers['layer_{}'.format(i+1)] = self.dense_layer(self.deep_layers['layer_{}'.format(i)],self.deep_size[i],name='layer_{}'.format(i))
            self.deep_output = self.deep_layers['layer_{}'.format(self.deep_layer_nums-1)]
            
            # loss and optimitizer
            self.feature_out = tf.concat([self.deep_output,self.fm_output], axis = 1)
            self.out = self.dense_layer(self.feature_out,1,activation=tf.nn.sigmoid,name="output")
           
            self.loss = tf.losses.log_loss(predictions=self.out,labels=self.labels)
            self.print = tf.print("loss:",self.out,output_stream=sys.stdout)
            if self.l2_reg > 0 :
                 self.loss += tf.losses.get_regularization_loss()
            
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
            
            #init
            self.saver = tf.train.Saver()
            self.sess = tf.Session()
            init = tf.global_variables_initializer()
            self.sess.run(init)
    
    def fit_on_batch(self,x_i,x_v,labels):
        feed_dict = {
            self.feat_index:x_i,
            self.feat_value:x_v,
            self.labels:labels
        }
        loss, _,out= self.sess.run([self.loss,self.optimizer,self.out],feed_dict = feed_dict)
        print(out,loss)
        return loss
    
    def shuffle_data(self,X_i,X_v,Y):
        now_state = np.random.get_state()
        np.random.shuffle(X_i)
        np.random.set_state(now_state)
        np.random.shuffle(X_v)
        np.random.set_state(now_state)
        np.random.shuffle(Y)
        
    def get_batch(self,X_i,X_v,Y,batch_size,step):
        pos_s = step * batch_size
        pos_e = (step+1) * batch_size
        # numpy slice 越界会自动选择到最后一个
        return X_i[pos_s:pos_e],X_v[pos_s:pos_e],Y[pos_s:pos_e]
    
    def evalute(self, x_v,x_i,y_label):
        y_pred = self.predict(x_i,x_v)
        return self.eval_metric(y_label,y_pred)
    
    def predict(self,x_v,x_i):
        
        
    
    def fit(self,X_i,X_v,Y,
           Xi_valid=None,Xv_valid=None,Y_valid=None):
        X_i = X_i.copy(); X_v=X_v.copy(); Y = Y.copy()
        for epoch in range(self.epochs):
            batch_nums = np.ceil(X_i.shape[0]/self.batch_size).astype(np.int64)
            if shuffle:
                self.shuffle_data(X_i,X_v,Y)
            total_loss = 0
            for step in range(batch_nums):
                x_i,x_v,y = self.get_batch(X_i,X_v,Y,self.batch_size,step)
                loss = self.fit_on_batch(x_i,x_v,y)
                total_loss += loss
                if(step % 100 == 0):
                    tf.logging.info("step={},loss={}".format(step,total_loss/(100*self.batch_size)))
                    total_loss = 0
                
            
       

In [109]:
fmmodel = DeepFM(**config) 
fmmodel.fit_on_batch(x_i[0:10],x_v[0:10],y[0:10].reshape([-1,1]))

[[0.6993106 ]
 [0.64715344]
 [0.6556818 ]
 [0.68481594]
 [0.7975112 ]
 [0.76910645]
 [0.5599257 ]
 [0.68055105]
 [0.7085018 ]
 [0.56484705]] 1.4833128


1.4833128

In [103]:
y.shape

(9489162,)

In [30]:
x_i.shape

(9489162, 24)

In [22]:
reload(deepfm)
model = deepfm.DeepFM(4260,24)


#params: 45637


In [19]:
model.fit_on_batch(x_i[0:10],x_v[0:10],y[0:10].reshape([-1,1]))

1.6118095

In [ ]:
fmmodel.fit(x_i[0:10000],x_v[0:10000],y[0:10000].reshape([-1,1]))

In [39]:
x_i.shape

(9489162, 24)

In [111]:
tf.placeholder?

Signature: tf.placeholder(dtype, shape=None, name=None)
Docstring:
Inserts a placeholder for a tensor that will be always fed.

**Important**: This tensor will produce an error if evaluated. Its value must
be fed using the `feed_dict` optional argument to `Session.run()`,
`Tensor.eval()`, or `Operation.run()`.

For example:

```python
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.Session() as sess:
  print(sess.run(y))  # ERROR: will fail because x was not fed.

  rand_array = np.random.rand(1024, 1024)
  print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.
```

@compatibility(eager)
Placeholders are not compatible with eager execution.
@end_compatibility

Args:
  dtype: The type of elements in the tensor to be fed.
  shape: The shape of the tensor to be fed (optional). If the shape is not
    specified, you can feed a tensor of any shape.
  name: A name for the operation (optional).

Returns:
  A `Tensor` that may be used as a handle for fee